# Initial Input

## Directory with the Object Locator Files

In [ ]:
#DIRECTORY = "/mnt/basestar/UNI/Masterarbeit/data/21-09-20/"
DIRECTORY = "../../data/21-09-20/"


## Analysis Parameters

In [ ]:
DENOIS_WEIGHT = 0
DECONV_ITERATIONS = 0
LDA_REF_OVERRIDE = 0

LDA_MIN = 350
LDA_MAX = 850


# Environment Setup

## Basic Utilities

In [ ]:
import sys
import os

## Maths

In [ ]:
import numpy as np
pi = np.pi

## Plotting

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

from util import plotstyle, devices, objects, tdms, calibration_persistent_data_path


In [ ]:
plotstyle.load('screen_dark')

# List known Devices, show Correction Spectra

In [ ]:
DEVS = devices.load_all()
for did in DEVS:
    print(did + ":\t" + DEVS[did].descr_str())

In [ ]:
fig = plt.figure(figsize=(12,8), dpi=100)

axs = fig.add_gridspec(1, 1)

ax = fig.add_subplot(axs[0, 0])

for did in DEVS:
    dev = DEVS[did]
    LDA = np.linspace( np.maximum(dev.ldamin, LDA_MIN), 
                       np.minimum(dev.ldamax, LDA_MAX), 
                       400 )
    spec, err = dev.evaluate(LDA)
    ax.fill_between( LDA, spec-err, spec+err, alpha=plotstyle.err_alpha(), )#color=line[1] )
    ax.plot( LDA, spec, lw=2, label=dev.descr_str() )#color=line[1] )
    
ax.set_title("Characteristic Spectra of the Optical Components")
ax.set_xlabel('$\lambda$ [nm]')
ax.set_ylabel('Relative Efficiency')

ax.legend()

ax.grid()

plt.tight_layout()
plt.show()

# Read the Object Locator Files

In [ ]:
object_files = [ "{d}/{fn}".format(d=DIRECTORY, fn=f) for f in os.listdir( DIRECTORY ) if f.endswith(".json") ]

object_files

In [ ]:
from util import objects
import json

object_descriptors = []

for fn in object_files:
    with open(fn, 'r') as file:
        object_descriptors.append( objects.Descriptor().from_dict( json.loads(file.read()) ) )

## Load Video Data

In [ ]:
OBJECTS = []
for d in object_descriptors:
    OBJECTS.append( objects.Object() )
    OBJECTS[-1].descriptor = d
    OBJECTS[-1].video = tdms.VideoSeries().load( d.videos )
    

In [ ]:
for o in OBJECTS:
    print( "{w}x{h} px,\t{f} Frames".format( w=o.video.width, h=o.video.height, f=o.video.frames ) )

## Apply the Calibration

In [ ]:
import pickle

with open( "{d}/calibration.pickle".format( d=calibration_persistent_data_path ), 'rb' ) as importfile:
    calibration = pickle.loads( importfile.read() )

In [ ]:
for o in OBJECTS:
    o.descriptor.ldaref = 532.0
    o.descriptor.sref = calibration['intercept'] + (-1.0)*o.descriptor.angle*calibration['slope']
    # TODO: divide sref by binning
    o.descriptor.sref /= o.video.binning
    

# ROI (Testing)

In [ ]:
#MAX_LDA = 900

In [ ]:
for o in OBJECTS:
    o.gen_roi( LDA_MAX, 1 )

In [ ]:
fig = plt.figure(figsize=(12,2*len(OBJECTS)), dpi=100)

axs = fig.add_gridspec(len(OBJECTS), 1)

for i in range(len(OBJECTS)):
    ax = fig.add_subplot(axs[ i , : ])

    ax.imshow(np.mean( OBJECTS[i].roi(), axis=0 ), 
              extent=OBJECTS[i].roi_extent(),
              cmap=plotstyle.cmap('m'))

    secxax = ax.secondary_xaxis(location='top', 
                                functions=(lambda x: x*OBJECTS[i].descriptor.ldaref/OBJECTS[i].descriptor.sref, 
                                           lambda x: x*OBJECTS[i].descriptor.sref/OBJECTS[i].descriptor.ldaref))
    secxax.set_xlabel('$\lambda_\mathrm{corresp.}$ [nm]')

    ax.set_xlabel('x [px]')
    ax.set_ylabel('y [px]')
    ax.set_title( OBJECTS[i].descriptor.particle.descr_str() )

plt.tight_layout()
plt.show()

In [ ]:
#BG = []
for o in OBJECTS:
    o.subtract_background()

In [ ]:
fig = plt.figure(figsize=(12,2*len(OBJECTS)), dpi=100)

axs = fig.add_gridspec(len(OBJECTS), 1)

for i in range(len(OBJECTS)):
    ax = fig.add_subplot(axs[ i , : ])
    
    #imgmax = np.max( np.log( np.mean( OBJECTS[i].roi(), axis=0 ) ) )

    ax.imshow(np.mean( OBJECTS[i].roi(), axis=0 ), 
              extent=OBJECTS[i].roi_extent(),
              cmap=plotstyle.cmap('m'))

    secxax = ax.secondary_xaxis(location='top', 
                                functions=(lambda x: x*OBJECTS[i].descriptor.ldaref/OBJECTS[i].descriptor.sref, 
                                           lambda x: x*OBJECTS[i].descriptor.sref/OBJECTS[i].descriptor.ldaref))
    secxax.set_xlabel('$\lambda_\mathrm{corresp.}$ [nm]')

    ax.set_xlabel('x [px]')
    ax.set_ylabel('y [px]')
    ax.set_title( OBJECTS[i].descriptor.particle.descr_str() )

plt.tight_layout()
plt.show()